# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

#COTAÇÃO DOLAR

navegador.get('https://www.google.com.br/')
navegador.find_element('xpath','//*[@id="APjFqb"]').send_keys('Cotação dólar')
navegador.find_element('xpath','//*[@id="APjFqb"]').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

#COTAÇÃO EURO

navegador.get('https://www.google.com.br/')
navegador.find_element('xpath','//*[@id="APjFqb"]').send_keys('Cotação euro')
navegador.find_element('xpath','//*[@id="APjFqb"]').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

#COTAÇÃO OURO

navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value').replace(',','.')
print(cotacao_ouro)

navegador.quit()

4.8618999999999994
5.246549218
305.76


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [4]:
# Atualizando a cotação
df_cotacao = pd.read_excel("Produtos.xlsx")
df_cotacao.loc[df_cotacao['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
df_cotacao.loc[df_cotacao['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
df_cotacao.loc[df_cotacao['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)
df_cotacao

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,4.861900,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.246549,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,4.861900,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,4.861900,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.246549,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,4.861900,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,305.760000,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [16]:
# Atualizando coluna com o preço de compra
df_cotacao['Preço de Compra'] = df_cotacao['Cotação'] * df_cotacao['Preço Original']
# Atualizando coluna com o preço de venda
df_cotacao['Preço de Venda'] = df_cotacao['Preço de Compra'] * df_cotacao['Margem']
# Formatando a Cotação
df_cotacao['Preço de Venda'] = df_cotacao['Preço de Venda'].map('R${:.2f}'.format)
df_cotacao

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,4.861900,4861.851381,1.40,R$6806.59
1,Carro Renault,4500.00,Euro,5.246549,23609.471481,2.00,R$47218.94
2,Notebook Dell,899.99,Dólar,4.861900,4375.661381,1.70,R$7438.62
3,IPhone,799.00,Dólar,4.861900,3884.658100,1.70,R$6603.92
4,Carro Fiat,3000.00,Euro,5.246549,15739.647654,1.90,R$29905.33
5,Celular Xiaomi,480.48,Dólar,4.861900,2336.045712,2.00,R$4672.09
6,Joia 20g,20.00,Ouro,305.760000,6115.200000,1.15,R$7032.48


### Agora vamos exportar a nova base de preços atualizada

In [7]:
df_cotacao.to_excel('Produtos_novo.xlsx', index=False)